In [93]:
from sentence_transformers import SentenceTransformer
import plotly.express as px
import pandas as pd

from sklearn.metrics import completeness_score, v_measure_score,normalized_mutual_info_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from code import utils,modeling
import nltk,re,time,umap,collections
from hdbscan import HDBSCAN
from sentence_transformers import models
from sklearn.preprocessing import normalize
from transformers import AutoModel, AutoTokenizer
from torch import nn
import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
client = MlflowClient()

#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
%reload_ext utils
%reload_ext modeling

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
X = utils.load_labeled_data()
y= X["labels"]
X.drop(["labels","site","entities_analyzed","updated_at"],inplace=True,axis="columns")
y_summarized_clusters = dict(collections.Counter(y))

X.head(1)

,headline,seo_title,text,created_at_date,created_at
article_uid,,,,,
5ed1398149006b863542719239e660f51f2bfb43eae5954d5a26b0596cbb54ae,US Open: Als Alexander Zverev seinen Eltern da...,"Als er seinen Eltern dankte, kamen ihm die Tränen",Was für ein Drama! Was für Emotionen! Alexande...,9/14/20,2020-09-14 11:56:15


In [43]:
#experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
mlflow.set_experiment("TT - Clustering")

embeddings = modeling.get_sentence_embeddings(pdf)
pdf = utils.preprocess_text(X,"text")

In [96]:

for N_COMPONENTS in [2,4,16,64,128,196,256,320,384,512,640]:
    for min_cluster_size in range(2, 4):    
        with mlflow.start_run():
            #ctr=0

            mlflow.log_param(key="min_cluster_size",value=min_cluster_size)
            mlflow.log_param(key="N_COMPONENTS",value=N_COMPONENTS)            
            results,cluster_labels = modeling.cluster_and_reduce(embeddings,n_components_clustering=N_COMPONENTS,min_cluster_size=min_cluster_size, min_samples= 3,alpha= 0.88)
            mlflow.log_metric(key="completeness_score", value=completeness_score(cluster_labels,y.values))
            mlflow.log_metric(key="v_measure_score", value=v_measure_score(cluster_labels,y.values))
            mlflow.log_metric(key="normalized_mutual_info_score", value=v_measure_score(cluster_labels,y.values))
            mlflow.log_metric(key="normalized_mutual_info_score", value=v_measure_score(cluster_labels,y.values))
            
            summarized_clusters = dict(collections.Counter(cluster_labels))
            try:
                mlflow.log_metric(key="outliers_ratio", value=(summarized_clusters[-1]/len(cluster_labels)))
            except Exception as err:
                mlflow.log_metric(key="outliers_ratio", value=0)

            mlflow.log_metric(key="unique_cluters", value=len(summarized_clusters.items()))
            mlflow.log_metric(key="unique_cluters_ratio", value=(len(summarized_clusters.items())-1)/len(y_summarized_clusters.items()))
            results = utils.link_to_raw_data(results,X,cluster_labels)
            
            modeling.scatter_plot(results,save_fig=True)
            mlflow.log_artifact("./tmp_scatter_plot.html")


>> Reducing dimensionality from 1536 to 128 ...
>> Clustering...
>> --- Done in 3.6 seconds ---
Outliers: 5 | Clustered: 234 | 0.9790794979079498 
 Cluster count: 18 
>> Reducing dimensionality from 1536 to 128 ...
>> Clustering...
>> --- Done in 3.9 seconds ---
Outliers: 1 | Clustered: 238 | 0.99581589958159 
 Cluster count: 14 
>> Reducing dimensionality from 1536 to 256 ...
>> Clustering...
>> --- Done in 4.2 seconds ---
Outliers: 1 | Clustered: 238 | 0.99581589958159 
 Cluster count: 14 
>> Reducing dimensionality from 1536 to 256 ...
>> Clustering...
>> --- Done in 3.6 seconds ---
Outliers: 4 | Clustered: 235 | 0.9832635983263598 
 Cluster count: 15 
>> Reducing dimensionality from 1536 to 384 ...
>> Clustering...
>> --- Done in 3.7 seconds ---
Outliers: 2 | Clustered: 237 | 0.9916317991631799 
 Cluster count: 15 
>> Reducing dimensionality from 1536 to 384 ...
>> Clustering...
>> --- Done in 4.4 seconds ---
Outliers: 2 | Clustered: 237 | 0.9916317991631799 
 Cluster count: 16 
>>

In [72]:
results,cluster_labels = modeling.cluster_and_reduce(embeddings,n_components_clustering=500,
                                                     min_cluster_size=2, min_samples= 3,alpha= 0.88)
completeness_score(cluster_labels,y.values), v_measure_score(cluster_labels,y.values)


--- Embedding dimension 1536
--- 239 Documnets encoded 37.68475890159607 seconds ---
>> Reducing dimensionality from 1536 to 500 ...
>> Clustering...
>> --- Done in 10.7 seconds ---


(0.921907445792243, 0.8348861954499294)